## `requirements.txt`

In [1]:
!cat requirements.txt

jupyterlab==4.0.2
tokenizers==0.7.0
torch==2.0.1
transformers==2.11.0


In [2]:
!pip list | grep "^tokenizers\|^transformers\|^torch\>"

tokenizers               0.7.0
torch                    2.0.1
transformers             2.11.0


## Data Source

In [3]:
from pathlib import Path

data_dir = Path("../../data/")
assert data_dir.is_dir()

In [4]:
URL = "https://cdn-datasets.huggingface.co/EsperBERTo/data/oscar.eo.txt"
file_name = URL.split("/")[-1]
file_name

'oscar.eo.txt'

No-progress-bar request

progress-bar request

In [5]:
file_path = data_dir/file_name

if file_path.exists():
    pass
else:
    import requests
    from tqdm.auto import tqdm

    response = requests.get(URL, stream=True)
    total_size_in_bytes= int(response.headers.get("content-length", 0))
    #print(f'{total_size_in_bytes = :,}')
    block_size = 1024
    progress_bar = tqdm(total=total_size_in_bytes, unit="iB", unit_scale=True)
    with open(file_path, "wb") as file:
        for data in response.iter_content(block_size):
            progress_bar.update(len(data))
            file.write(data)
    progress_bar.close()

    if total_size_in_bytes != 0 and progress_bar.n != total_size_in_bytes:
        print("ERROR, something went wrong")
        file_path

## Tokenizers

The next cell will take some time to run, and w/o progress bar, even though there is an input
argument `show_progress` which defaults to `True` for the method `train`.

**Rmk.** According to [this issue from GitHub](https://github.com/huggingface/tokenizers/issues/157),
it seems that the **progress bar works fine** when running as a Python script **in terminal**, but **fails in notebook**.

In [6]:
tokenizer_dir = Path("EsperBERTo")
tokenizer_dir.mkdir(exist_ok=True)

In [7]:
from tokenizers import ByteLevelBPETokenizer

no_trained_tokenizer = list(tokenizer_dir.glob("*")) == []
if no_trained_tokenizer:
    tokenizer = ByteLevelBPETokenizer()
    tokenizer.train(files=[str(file_path)], vocab_size=52_000, min_frequency=2, special_tokens=[
        "<s>",
        "<pad>",
        "</s>",
        "<unk>",
        "<mask>",
    ])
    tokenizer.save(str(tokenizer_dir))

### Reload tokenizer and add `post_processor`

In [8]:
from tokenizers.implementations import ByteLevelBPETokenizer

tokenizer = ByteLevelBPETokenizer(
    str(tokenizer_dir/"vocab.json"),
    str(tokenizer_dir/"merges.txt"),
)

In [9]:
encoding = tokenizer.encode("Mi estas Julien.")
encoding.tokens

['Mi', 'Ġestas', 'ĠJuli', 'en', '.']

In [10]:
from tokenizers.processors import BertProcessing

# tokenizer._tokenizer.post_processor = BertProcessing(
#     (tokenizer.eos_token, tokenizer.eos_token_id)
#     (tokenizer.bos_token, tokenizer.bos_token_id)
# )
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [11]:
encoding = tokenizer.encode("Mi estas Julien.")
encoding.tokens

['<s>', 'Mi', 'Ġestas', 'ĠJuli', 'en', '.', '</s>']

In [12]:
hasattr(tokenizer, "is_fast")

False

### Fast tokenizer

**(?)** But won't this make our previously defined `post_processor` disappear?  

In [13]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("./EsperBERTo", max_len=512)

/home/phunc20/.config/miniconda3/envs/lm_from_scratch/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/phunc20/.config/miniconda3/envs/lm_from_scratch/lib/python3.8/site-packages/transformers/tokenization_utils.py:828: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  warnings.warn(


In [14]:
hasattr(tokenizer, "is_fast")

True

In [15]:
tokenizer.is_fast

True

## Dataset

In [16]:
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path=file_path,
    #block_size=128,
    block_size=64,
)

In [17]:
dataset

**(?)** `block_size` is the context length?  

### Data Collator

In [18]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

## Train

In [19]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [20]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)

### Package problem
`Python==3.8.16` with `transformers==2.11.0` cannot import `RobertaForMaskedLM`
```bash
ImportError: cannot import name 'RobertaForMaskedLM' from 'transformers' (/home/phunc20/.config/miniconda3/envs/lm_from_scratch/lib/python3.8/site-packages/transformers/__init__.py)
```

**Solution.** This is because you **need to install `torch`**. I have already included `torch` in `requirements.txt`
to avoid this.

In [21]:
import torch

torch.cuda.empty_cache()

In [22]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir=str(tokenizer_dir),
    overwrite_output_dir=True,
    #num_train_epochs=1,
    max_steps=1_000,
    #per_gpu_train_batch_size=1,
    per_device_train_batch_size=1,
    save_steps=100,
    logging_steps=100,
    save_total_limit=3,
    #fp16=True,
    # prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [23]:
trainer.train()

Iteration:   0%|                                                                              | 99/974545 [01:03<174:08:53,  1.55it/s]

{"loss": 9.480108938217164, "learning_rate": 4.5e-05, "epoch": 0.00010261198815857657, "step": 100}



Iteration:   0%|                                                                             | 199/974545 [02:08<173:03:56,  1.56it/s]

{"loss": 8.751515731811523, "learning_rate": 4e-05, "epoch": 0.00020522397631715314, "step": 200}



Iteration:   0%|                                                                             | 299/974545 [03:14<173:53:15,  1.56it/s]

...

{"loss": NaN, "learning_rate": 2e-05, "epoch": 0.0006156719289514594, "step": 600}



Iteration:   0%|                                                                             | 699/974545 [07:38<175:02:06,  1.55it/s]

{"loss": NaN, "learning_rate": 1.5e-05, "epoch": 0.0007182839171100359, "step": 700}



Epoch:   0%|                                                                                                    | 0/1 [08:07<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
model.save_pretrained(tokenizer_dir)

In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model=str(tokenizer_dir),
    tokenizer=str(tokenizer_dir),
)

In [ ]:
# The sun <mask>.
fill_mask("La suno <mask>.")

In [ ]:
# This is the beginning of a beautiful <mask>.
fill_mask("Jen la komenco de bela <mask>.")